### Importo le cartelle da drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import cv2

### Verifico la GPU

In [1]:
!nvidia-smi

"nvidia-smi" non � riconosciuto come comando interno o esterno,
 un programma eseguibile o un file batch.


###  Installo YOLOv8

In [ ]:
!pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

from IPython.display import display, Image

from ultralytics import YOLO